# init

In [2]:
# import
import aiohttp
import pkuseg
import pymongo
import zhconv
from types import TracebackType
from typing import Dict, Optional, Type
from aip import AipNlp
from motor import motor_asyncio
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

    
# CONFIG
if platform.system() == 'Windows':
    # CODE_PATH
    CODE_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/code')
    DATA_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/data')
    BERT_PATH = os.path.join(os.environ['onedrive'], 'Data/BERT')
    ERNIE_PATH = os.path.join(os.environ['onedrive'], 'Data/baidu-paddle/LARK/ERNIE')
    ERNIE_LAC_PATH=os.path.join(os.environ['onedrive'], 'Data/baidu-paddle/models/PaddleNLP/lexical_analysis')
    %cd {CODE_PATH}
elif platform.system() == 'Linux':
    # CODE_PATH
    CODE_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/code'
    DATA_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/data'
    %cd {CODE_PATH}

# Baidu NLP
def get_token(accounts):
    '''get API tokens'''
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials"
    for account in accounts:
        # client_id 为官网获取的AK， client_secret 为官网获取的SK
        param = {'client_id': account['apiKey'],
                 'client_secret': account['secretKey']}
        resp = requests.get(url, param).json()
        yield resp['access_token']
        
accounts = [{'appId': '16105374', # ross.zhu@outlook.com
            'apiKey': 'zShobECEYX2B2CNrcz3QsM3I',
            'secretKey': 'xM8g1lkhTd8b7DYGv2H9j7g2hvthPGE5'},
           {'appId': '16218655', # 18868819096
            'apiKey': 'AG2ygGZbCPqDW25ZC6FinQRy',
            'secretKey': 'pcx8pFwjVhD0oGDsdm58bzt7Zm9zFpWk'},
           {'appId': '16226200', # 13566633705
            'apiKey': 'ymmGiPQoAoSF2KhcGVKNZEVj',
            'secretKey': '78oQXsRtdmRcINEGtP3aP2y4sKA7HerU'},
           {'appId': '16226243', # 13606586786
            'apiKey': 'jIy3T25rYR1Kj9BlCKPcHbYe',
            'secretKey': 'jORNEeu7Qnl6L1Hr07NomigmsUtXlloL'},
           {'appId': '16230746', # 李锐
            'apiKey': 't6sWsoIYinoGBgwbLiCopQHR',
            'secretKey': 'evq4UygxQ555LXzVzaDWQS2TfGNUHrYf'},
           {'appId': '16307961', # 俞嘉炜
            'apiKey': 'TKIEhGe1fIY89VlYm8yjuDZW',
            'secretKey': '6kDBxZnGZEsK4y9ntg94VOBSIjLS0UQs'},
           {'appId': '16305932', # 芳菲
            'apiKey': 'Y7NjBe2sqhfIjSIaB51wvlHH',
            'secretKey': 'lqLW2XeSj0xBvCbUASh0E2wuMmW6e4ox'},
           {'appId': '16305931', # 小毫生
            'apiKey': 'D2GGEWS80KsU9mtft1WnuR0N',
            'secretKey': '8FbvLeGD5aXLL6bpKSEZoG7CRt1Q9oqo'},
           {'appId': '16306399', # 忻小宇
            'apiKey': 'E89qU1OMrwvyanH7xixkFVrR',
            'secretKey': 'ljPLgCtzOKLZuG56DPX9hvrVFHRsWHqf'},
           {'appId': '16306475', # 杨老板
            'apiKey': 'yehP04KXoWADTGUyLF8WK0hN',
            'secretKey': 'jfp3wxqupbWfEoNi7L316LdfbUmDmRUo'},
           ]
tokens = list(get_token(accounts))
baidu_clients = [AipNlp(**account) for account in accounts] 

# MongoDB-pymongo
pymongo_client = MongoClient('localhost', 27018)
db = pymongo_client.exchange
coll_emb_bd = db.emb_bd
coll_vocab_without_emb = db.vocab_without_emb
coll_vocab_to_request = db.vocab_to_request
coll_vocab_pos = db.vocab_pos
coll_vocab_ne = db.vocab_ne
coll_text_seg_bd = db.text_seg_bd
coll_text_seg_pku = db.text_seg_pku
coll_title_seg_bd = db.title_seg_bd
coll_title_seg_pku = db.title_seg_pku
coll_test = db.test

C:\Users\rossz\OneDrive\STN\10-FirstYear\code


# create requests

In [ ]:
for word in vocab_dup:
    with capture_output():
        coll_vocab_to_request.delete_one({'word':word})

In [3]:
%%time
# vocab_last
vocab_with_emb = set(doc['word'] for doc in coll_emb_bd.find(projection={'word':1, '_id':0}))
print(f'vocab_with_emb: {len(vocab_with_emb)}')
   
# vocab_without_emb
vocab_without_emb = set(doc['word'] for doc in coll_vocab_without_emb.find(projection={'word':1, '_id':0}))
print(f'vocab_without_emb: {len(vocab_without_emb)}')

# delete dup in coll_vocab_to_request
vocab_to_request = set(doc['word'] for doc in coll_vocab_to_request.find(projection={'word':1, '_id':0}))
vocab_dup =  (vocab_to_request & vocab_with_emb) | (vocab_to_request & vocab_without_emb)

for word in vocab_dup:
    with capture_output():
        coll_vocab_to_request.delete_one({'word':word})

# vocab_request
vocab_request = vocab_to_request - vocab_with_emb - vocab_without_emb

# # vocab from GBK
# gbk_path = os.path.expanduser('~')+'/OneDrive/Data/gbk-list.txt'
# gbk_list = []
# with open(gbk_path, encoding='utf-8') as f:
#     for line in f:
#         line = line.replace('\n', '').strip()
#         gbk_list.append(line)
# vocab_request = set(gbk_list) - vocab_without_emb - vocab_last

print(f'vocab_request: {len(vocab_request)}')

vocab_with_emb: 396171
vocab_without_emb: 7636551
vocab_request: 898321
Wall time: 35.2 s


# get emb

In [ ]:
async def get_emb(word, token):
    base_url = 'https://aip.baidubce.com/rpc/2.0/nlp/v2/word_emb_vec'
    url_params = {'access_token': token,
                  'charset': 'UTF-8'}
    request_text = {'word': word}
    headers = {'Content-Type': 'application/json'}

    async with aiohttp.ClientSession() as session:
        async with session.request('POST', url=base_url, json=request_text, headers=headers, params=url_params) as resp:
            response = await resp.json()
            return response

async def request_one(word, token, bucket, download_delay):
    global count_all, last_count_all, count_success,\
        count_non_exist, count_input_too_long, count_input_empty,\
        count_limit_reached, count_unknow_error,\
        count_mongo_error, count_duplicate, t0
    try:
        await asyncio.sleep(download_delay)
        async with bucket:
                response = await get_emb(word, token)
        if 'error_code' in response:
            if response['error_code'] == 18: # limit reached
                count_limit_reached += 1
            elif response['error_code'] == 282131: # input too long
                count_input_too_long += 1
            elif response['error_code'] == 282134: # input empty
                count_input_empty += 1
            elif response['error_code'] == 282300: # non-existing
                count_non_exist += 1
                with capture_output() as captured:
                    coll_vocab_without_emb.insert_one({'word': word})
            else:
                print(f'Other error_code: word:"{word}", response:"{response}"')
                count_unknow_error += 1
        elif 'vec' in response:
            vec = response['vec']
            with capture_output() as captured:
                assert type(vec) is list, 'type(vec) is not list: {word}'
                assert len(vec) == 1024, 'len(vec) should be 1024'
                coll_emb_bd.insert_one({'word': word, 'vec': vec})
                count_success += 1
    except DuplicateKeyError:
        count_duplicate += 1
    except:
        print(f"Unknow exception: {sys.exc_info()[:-1]}, {Now()}")
        count_unknow_error += 1
    finally:
        if count_all % 10000 == 0:
            te = time.time()
            print(f'{count_all}/{len(vocab_request)}, {Now()} ({te-t0:.1f}s / {(count_all-last_count_all)/(te-t0):.1f}QPS)')
            print(f'success:{count_success} non_existing:{count_non_exist} duplicate:{count_duplicate} limit_reached:{count_limit_reached} input_too_long:{count_input_too_long} input_empty:{count_input_empty} unkown_error:{count_unknow_error}')
            print('---------------------')
            t0 = time.time()
            count_success, count_non_exist, count_duplicate, count_input_too_long, count_input_empty,\
            count_limit_reached, count_unknow_error, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0
            last_count_all = count_all
    
async def download_many(tokens, vocab_request, max_concur_req, download_delay):
    global count_all
    tokens_iter = itertools.cycle(tokens)
    # restrict concurrenty
    bucket = AsyncLeakyBucket(max_concur_req, 1)
    # generate tasks
    print(f'start creating tasks... {Now()}')
    tasks = []
    for word in vocab_request:
        task = request_one(word, next(tokens_iter), bucket, download_delay)
        tasks.append(task)
    print(f'tasks succesfully created {Now()}')
        
    # schedule tasks
    for future in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        count_all += 1
        await future

# run the main
# vocab_request = [doc['word'] for doc in coll_vocab_to_request.find(projection={'word':1, '_id':0})][:300]
# coll_test.delete_many({})
t0 = time.time()
count_all, last_count_all, count_success, count_non_exist, count_input_too_long, count_input_empty, count_limit_reached, count_unknow_error, count_mongo_error, count_duplicate = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
asyncio.run(download_many(tokens, vocab_request, max_concur_req=32, download_delay=0))

start creating tasks... 08:08:44
tasks succesfully created 08:08:58


  1%|▍                                                                      | 5750/898321 [02:59<6:04:30, 40.81it/s]